In [1]:
# read in the matrix
import pandas as pd
import pickle

data = pd.read_pickle('dtm_stop.pkl')
data

,abased,abasedand,abba,abel,abelto,abia,abide,abidein,abideth,abidethere,...,youyou,zabulon,zacchaeus,zacchaeuswanted,zacharias,zeal,zebedee,zebedees,zelotes,zorobabel
John,0,0,0,0,0,0,9,1,4,0,...,0,0,0,0,0,1,0,0,0,0
Luke,2,0,0,1,0,1,2,0,0,0,...,1,0,3,1,8,0,0,0,1,1
Mark,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,3,0,0,0
Matthew,0,1,0,0,1,0,1,0,0,0,...,0,1,0,0,1,0,3,1,0,0


In [2]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse

In [3]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head()

,John,Luke,Mark,Matthew
abased,0,2,0,0
abasedand,0,0,0,1
abba,0,0,1,0
abel,0,1,0,0
abelto,0,0,0,1


In [4]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [5]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [6]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.028*"unto" + 0.010*"saith" + 0.008*"world" + 0.008*"know" + 0.007*"answered" + 0.006*"hath" + 0.006*"jews" + 0.005*"sent" + 0.005*"believe" + 0.004*"verily"'),
 (1,
  '0.005*"people" + 0.004*"day" + 0.004*"house" + 0.004*"took" + 0.004*"away" + 0.004*"heaven" + 0.004*"cast" + 0.004*"let" + 0.004*"did" + 0.004*"men"')]

In [8]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.005*"house" + 0.005*"people" + 0.005*"day" + 0.004*"saying" + 0.003*"great" + 0.003*"let" + 0.003*"tell" + 0.003*"good" + 0.003*"called" + 0.003*"john"'),
 (1,
  '0.000*"unto" + 0.000*"saying" + 0.000*"day" + 0.000*"did" + 0.000*"let" + 0.000*"sent" + 0.000*"know" + 0.000*"men" + 0.000*"people" + 0.000*"behold"'),
 (2,
  '0.016*"unto" + 0.005*"know" + 0.005*"answered" + 0.005*"saith" + 0.005*"world" + 0.004*"took" + 0.004*"heaven" + 0.004*"away" + 0.004*"sent" + 0.004*"did"')]

In [9]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.005*"took" + 0.005*"heaven" + 0.005*"away" + 0.004*"cast" + 0.004*"unto" + 0.004*"did" + 0.004*"people" + 0.004*"men" + 0.004*"heard" + 0.004*"let"'),
 (1,
  '0.035*"unto" + 0.012*"saith" + 0.010*"world" + 0.009*"know" + 0.008*"answered" + 0.008*"hath" + 0.007*"jews" + 0.007*"sent" + 0.006*"believe" + 0.005*"verily"'),
 (2,
  '0.000*"unto" + 0.000*"away" + 0.000*"day" + 0.000*"people" + 0.000*"answered" + 0.000*"let" + 0.000*"saying" + 0.000*"sent" + 0.000*"cast" + 0.000*"heaven"'),
 (3,
  '0.005*"house" + 0.005*"people" + 0.005*"day" + 0.004*"saying" + 0.004*"great" + 0.003*"let" + 0.003*"tell" + 0.003*"good" + 0.003*"called" + 0.003*"john"')]

In [10]:
# These topics could be better. Is unto a topic? no
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [11]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('data_clean.pkl')
data_clean

,transcripts
John,title the gospel of john for readers editor li...
Luke,title the gospel of luke for readers editor li...
Mark,title the gospel of mark for readers editor li...
Matthew,title the gospel of matthew for readers editor...


In [13]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean.transcripts.apply(nouns))
data_nouns

,transcripts
John,gospel john readers editor lightheart release ...
Luke,gospel luke readers editor lightheart release ...
Mark,gospel mark readers editor lightheart release ...
Matthew,gospel matthew readers editor lightheart relea...


In [51]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said', 'unto', 'saith', 'hath', 'thou', 'ye', 'thee', 'hast', 'shalt',
                 'sayest', 'cometh', 'woe']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=list(stop_words))
data_cvn = cvn.fit_transform(data_nouns.transcripts)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names_out())
data_dtmn.index = data_nouns.index
data_dtmn

,abel,abide,abideth,abilene,ability,aboat,abode,abomination,aboutgalilee,aboutthee,...,youth,youyou,zabulon,zacchaeus,zacharias,zeal,zebedee,zebedees,zelotes,zorobabel
John,0,3,2,0,0,0,2,0,0,0,...,0,0,0,0,0,1,0,0,0,0
Luke,1,1,0,1,0,0,1,0,0,1,...,1,1,0,2,8,0,0,0,1,1
Mark,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,3,0,0,0
Matthew,0,0,0,0,1,0,0,0,1,0,...,1,0,1,0,1,0,3,1,0,0


In [52]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [53]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.044*"jesus" + 0.026*"man" + 0.019*"son" + 0.013*"god" + 0.012*"things" + 0.012*"lord" + 0.010*"disciples" + 0.008*"kingdom" + 0.008*"day" + 0.008*"house"'),
 (1,
  '0.042*"jesus" + 0.021*"man" + 0.020*"father" + 0.015*"world" + 0.013*"son" + 0.012*"god" + 0.012*"things" + 0.011*"jews" + 0.010*"disciples" + 0.008*"life"')]

In [54]:
# looking better
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.015*"jesus" + 0.006*"man" + 0.004*"boat" + 0.003*"scribes" + 0.003*"disciples" + 0.002*"damsel" + 0.002*"james" + 0.002*"desire" + 0.002*"sea" + 0.002*"bread"'),
 (1,
  '0.047*"jesus" + 0.028*"man" + 0.021*"son" + 0.014*"god" + 0.013*"things" + 0.013*"lord" + 0.011*"disciples" + 0.009*"kingdom" + 0.009*"day" + 0.008*"house"'),
 (2,
  '0.047*"jesus" + 0.024*"man" + 0.023*"father" + 0.017*"world" + 0.014*"son" + 0.014*"god" + 0.013*"things" + 0.013*"jews" + 0.011*"disciples" + 0.009*"life"')]

In [55]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.052*"jesus" + 0.028*"man" + 0.012*"disciples" + 0.011*"son" + 0.010*"things" + 0.010*"god" + 0.006*"house" + 0.006*"hand" + 0.006*"kingdom" + 0.006*"way"'),
 (1,
  '0.040*"jesus" + 0.021*"man" + 0.014*"son" + 0.011*"lord" + 0.011*"disciples" + 0.010*"father" + 0.010*"things" + 0.009*"kingdom" + 0.008*"heaven" + 0.008*"god"'),
 (2,
  '0.001*"jesus" + 0.001*"son" + 0.001*"man" + 0.001*"god" + 0.001*"lord" + 0.001*"disciples" + 0.001*"things" + 0.001*"father" + 0.000*"day" + 0.000*"house"'),
 (3,
  '0.047*"jesus" + 0.027*"man" + 0.023*"son" + 0.018*"god" + 0.016*"father" + 0.016*"things" + 0.012*"lord" + 0.010*"world" + 0.010*"disciples" + 0.009*"day"')]

In [56]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [57]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcripts.apply(nouns_adj))
data_nouns_adj

,transcripts
John,gospel john readers editor lightheart release ...
Luke,gospel luke readers editor lightheart release ...
Mark,gospel mark readers editor lightheart release ...
Matthew,gospel matthew readers editor lightheart relea...


In [58]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=list(stop_words), max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcripts)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names_out())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,abel,abia,abide,abideth,abilene,ability,able,ableto,aboat,abode,...,youwill,youyou,zabulon,zacchaeus,zacharias,zeal,zebedee,zebedees,zelotes,zorobabel
John,0,0,3,2,0,0,1,1,0,2,...,0,0,0,0,0,1,0,0,0,0
Luke,1,1,1,0,1,0,6,0,0,1,...,1,1,0,2,8,0,0,0,1,1
Mark,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,3,0,0,0
Matthew,0,0,0,0,0,1,9,0,0,0,...,0,0,1,0,1,0,3,1,0,0


In [59]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [60]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.006*"light" + 0.005*"law" + 0.004*"disciple" + 0.004*"works" + 0.004*"sin" + 0.003*"fathers" + 0.003*"philip" + 0.002*"love" + 0.002*"miracles" + 0.002*"thejews"'),
 (1,
  '0.004*"devils" + 0.003*"herod" + 0.003*"eye" + 0.003*"generation" + 0.003*"faith" + 0.003*"light" + 0.003*"elders" + 0.003*"rich" + 0.002*"vineyard" + 0.002*"angel"')]

In [61]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.004*"eye" + 0.003*"field" + 0.003*"talents" + 0.003*"devils" + 0.003*"light" + 0.003*"faith" + 0.003*"generation" + 0.003*"elders" + 0.003*"hypocrites" + 0.003*"herod"'),
 (1,
  '0.004*"devils" + 0.003*"herod" + 0.003*"rich" + 0.003*"generation" + 0.003*"james" + 0.003*"angel" + 0.003*"faith" + 0.003*"unclean" + 0.002*"cast" + 0.002*"light"'),
 (2,
  '0.007*"light" + 0.006*"law" + 0.005*"disciple" + 0.005*"works" + 0.005*"sin" + 0.004*"fathers" + 0.003*"philip" + 0.003*"love" + 0.003*"miracles" + 0.003*"thejews"')]

In [62]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.000*"devils" + 0.000*"light" + 0.000*"rich" + 0.000*"generation" + 0.000*"eye" + 0.000*"herod" + 0.000*"law" + 0.000*"angel" + 0.000*"faith" + 0.000*"james"'),
 (1,
  '0.008*"light" + 0.006*"law" + 0.005*"disciple" + 0.005*"works" + 0.005*"sin" + 0.004*"fathers" + 0.004*"philip" + 0.003*"love" + 0.003*"thejews" + 0.003*"miracles"'),
 (2,
  '0.004*"devils" + 0.004*"light" + 0.004*"eye" + 0.004*"generation" + 0.003*"faith" + 0.003*"angel" + 0.003*"herod" + 0.003*"worthy" + 0.003*"rich" + 0.003*"field"'),
 (3,
  '0.004*"james" + 0.003*"herod" + 0.003*"devils" + 0.003*"unclean" + 0.003*"elders" + 0.003*"cast" + 0.003*"damsel" + 0.002*"soul" + 0.002*"swine" + 0.002*"possible"')]

In [63]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=80)
ldana.print_topics()

[(0,
  '0.004*"rich" + 0.004*"devils" + 0.004*"angel" + 0.003*"light" + 0.003*"abraham" + 0.003*"generation" + 0.003*"women" + 0.003*"faith" + 0.003*"worthy" + 0.003*"herod"'),
 (1,
  '0.000*"taste" + 0.000*"andif" + 0.000*"question" + 0.000*"testament" + 0.000*"shining" + 0.000*"fields" + 0.000*"vessel" + 0.000*"greek" + 0.000*"spices" + 0.000*"present"'),
 (2,
  '0.006*"light" + 0.005*"law" + 0.004*"works" + 0.004*"disciple" + 0.003*"eye" + 0.003*"sin" + 0.003*"fathers" + 0.003*"field" + 0.002*"talents" + 0.002*"prison"'),
 (3,
  '0.004*"james" + 0.003*"unclean" + 0.003*"herod" + 0.003*"devils" + 0.003*"elders" + 0.003*"damsel" + 0.003*"cast" + 0.002*"possible" + 0.002*"heed" + 0.002*"swine"')]

In [64]:
# Let's take a look at which topics each transcript contains
corpus_transformed = ldana[corpusna]
list(zip([a for [(a,b)] in corpus_transformed], data_dtmna.index))

[(2, 'John'), (0, 'Luke'), (3, 'Mark'), (2, 'Matthew')]